In [2]:
%pip install gensim
import numpy as np

## Get yhe interactive tools for Matplotlib
#  matplotlib notebook 
%matplotlib inline 
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.decomposition import PCA

import gensim.downloader as api
from gensim.models import KeyedVectors


In [3]:
model = api.load("glove-wiki-gigaword-100")
print(type[model])

type[<gensim.models.keyedvectors.KeyedVectors object at 0x00000180CA0BAA50>]


In [4]:
model['bread']

array([-0.66146  ,  0.94335  , -0.72214  ,  0.17403  , -0.42524  ,
        0.36303  ,  1.0135   , -0.14802  ,  0.25817  , -0.20326  ,
       -0.64338  ,  0.16632  ,  0.61518  ,  1.397    , -0.094506 ,
        0.0041843, -0.18976  , -0.55421  , -0.39371  , -0.22501  ,
       -0.34643  ,  0.32076  ,  0.34395  , -0.7034   ,  0.23932  ,
        0.69951  , -0.16461  , -0.31819  , -0.34034  , -0.44906  ,
       -0.069667 ,  0.35348  ,  0.17498  , -0.95057  , -0.2209   ,
        1.0647   ,  0.23231  ,  0.32569  ,  0.47662  , -1.1206   ,
        0.28168  , -0.75172  , -0.54654  , -0.66337  ,  0.34804  ,
       -0.69058  , -0.77092  , -0.40167  , -0.069351 , -0.049238 ,
       -0.39351  ,  0.16735  , -0.14512  ,  1.0083   , -1.0608   ,
       -0.87314  , -0.29339  ,  0.68278  ,  0.61634  , -0.088844 ,
        0.88094  ,  0.099809 , -0.27161  , -0.58026  ,  0.50364  ,
       -0.93814  ,  0.67576  , -0.43124  , -0.10517  , -1.2404   ,
       -0.74353  ,  0.28637  ,  0.29012  ,  0.89377  ,  0.6740

In [5]:
model.most_similar('usa')

[('canada', 0.6544384956359863),
 ('america', 0.645224392414093),
 ('u.s.a.', 0.6184033751487732),
 ('united', 0.6017189621925354),
 ('states', 0.5970699191093445),
 ('australia', 0.5838716626167297),
 ('world', 0.5590084195137024),
 ('2010', 0.5580702424049377),
 ('2012', 0.5504006743431091),
 ('davis', 0.5464468002319336)]

In [7]:
result = model.most_similar(positive=['woman', 'king'], negative=['man'])
print(result)

[('queen', 0.7698540687561035), ('monarch', 0.6843381524085999), ('throne', 0.6755736470222473), ('daughter', 0.6594556570053101), ('princess', 0.6520534157752991), ('prince', 0.6517034769058228), ('elizabeth', 0.6464517712593079), ('mother', 0.631171703338623), ('emperor', 0.6106470823287964), ('wife', 0.6098655462265015)]


In [8]:
# Find words similar to 'computer'
similar_words = model.most_similar('computer')
print("Words similar to 'computer':", similar_words)

# Calculate similarity between two words
similarity_score = model.similarity('woman', 'man')
print("Similarity between 'woman' and 'man':", similarity_score)

# Get the vector representation of a word
word_vector = model['computer']
print("Vector representation of 'computer':", word_vector)

# Find words that are related in the same way as 'king' is to 'man'
analogy_result = model.most_similar(positive=['woman', 'king'], negative=['man'])
print("Words related to 'king' as 'woman' is to 'man':", analogy_result)

Words similar to 'computer': [('computers', 0.8751984238624573), ('software', 0.8373122215270996), ('technology', 0.7642159461975098), ('pc', 0.7366448640823364), ('hardware', 0.7290390729904175), ('internet', 0.7286775708198547), ('desktop', 0.7234441637992859), ('electronic', 0.7221828699111938), ('systems', 0.7197922468185425), ('computing', 0.7141730785369873)]
Similarity between 'woman' and 'man': 0.8323495
Vector representation of 'computer': [-1.6298e-01  3.0141e-01  5.7978e-01  6.6548e-02  4.5835e-01 -1.5329e-01
  4.3258e-01 -8.9215e-01  5.7747e-01  3.6375e-01  5.6524e-01 -5.6281e-01
  3.5659e-01 -3.6096e-01 -9.9662e-02  5.2753e-01  3.8839e-01  9.6185e-01
  1.8841e-01  3.0741e-01 -8.7842e-01 -3.2442e-01  1.1202e+00  7.5126e-02
  4.2661e-01 -6.0651e-01 -1.3893e-01  4.7862e-02 -4.5158e-01  9.3723e-02
  1.7463e-01  1.0962e+00 -1.0044e+00  6.3889e-02  3.8002e-01  2.1109e-01
 -6.6247e-01 -4.0736e-01  8.9442e-01 -6.0974e-01 -1.8577e-01 -1.9913e-01
 -6.9226e-01 -3.1806e-01 -7.8565e-01

In [4]:
import numpy as np
from typing import List, Dict, Tuple
import faiss
from sentence_transformers import SentenceTransformer

class VectorSimilaritySearch:
    def __init__(self, dimension: int = 384):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.index = faiss.IndexFlatL2(dimension)
        self.text_lookup: Dict[int, str] = {}
        self.current_id = 0
    
    def add_texts(self, texts: List[str]) -> None:
        # Convert texts to vectors
        vectors = self.model.encode(texts)
        
        # Add to FAISS index
        for i, text in enumerate(texts):
            vector = vectors[i].reshape(1, -1)
            self.index.add(vector.astype('float32'))
            self.text_lookup[self.current_id] = text
            self.current_id += 1
    
    def search(self, query: str, k: int = 5) -> List[Tuple[str, float]]:
        # Convert query to vector
        query_vector = self.model.encode([query])[0].reshape(1, -1)
        
        # Search index
        distances, indices = self.index.search(
            query_vector.astype('float32'), k
        )
        
        # Format results
        results = []
        for idx, dist in zip(indices[0], distances[0]):
            if idx != -1:
                similarity = 1 / (1 + dist)  # Convert distance to similarity
                results.append((self.text_lookup[idx], similarity))
        
        return results

    def batch_search(
        self, 
        queries: List[str], 
        k: int = 5
    ) -> List[List[Tuple[str, float]]]:
        # Convert queries to vectors
        query_vectors = self.model.encode(queries)
        
        # Search index
        distances, indices = self.index.search(
            query_vectors.astype('float32'), k
        )
        
        # Format results
        all_results = []
        for query_indices, query_distances in zip(indices, distances):
            results = []
            for idx, dist in zip(query_indices, query_distances):
                if idx != -1:
                    similarity = 1 / (1 + dist)
                    results.append((self.text_lookup[idx], similarity))
            all_results.append(results)
            
        return all_results

In [7]:

searcher = VectorSimilaritySearch()

# Add documents
docs = [
    "Neural networks are powerful",
    "Machine learning transforms industries",
    "Deep learning revolutionizes AI",
    "Computer vision analyzes images"
]
searcher.add_texts(docs)

# Single search
results = searcher.search("AI and neural networks")
print(f"Top matches: {results}")

# Batch search
queries = ["machine learning", "computer vision"]
batch_results = searcher.batch_search(queries)
print(f"Batch results: {batch_results}")

Top matches: [('Neural networks are powerful', 0.6104603325421382), ('Deep learning revolutionizes AI', 0.4998297377257882), ('Machine learning transforms industries', 0.435180225117334), ('Computer vision analyzes images', 0.43361149034720936)]
Batch results: [[('Machine learning transforms industries', 0.5065744528042764), ('Neural networks are powerful', 0.4958963531145656), ('Deep learning revolutionizes AI', 0.45741436316591555), ('Computer vision analyzes images', 0.4389900402326039)], [('Computer vision analyzes images', 0.6882277264826985), ('Neural networks are powerful', 0.4482184873626295), ('Machine learning transforms industries', 0.4222419189292866), ('Deep learning revolutionizes AI', 0.41494644719694507)]]
